# Продолжение работы с ZDNABERT локально, а не в колабе

In [22]:
import pandas as pd
from tqdm import tqdm

In [25]:
results = pd.DataFrame(columns = ["Scaffold", "Start", "End"])
with open("text_predictions_zdnabert.txt", "r") as f:
    scaffold = ""
    lines = f.readlines()
    for line in tqdm(lines):
        if line == "genomic\n":
            continue
        if line.startswith("NC") or line.startswith("NW"):
            scaffold = line[:-1]
            continue
        if line.split()[0] == "start":
            continue
        if len(line.split()) < 2:
            start = int(line.split()[0][0:8])
            end = int(line.split()[0][8:16])
        else:
            start, end = int(line.split()[0]), int(line.split()[1])
        results.loc[len(results)] = [scaffold, start, end]
results

100%|██████████| 90190/90190 [01:36<00:00, 932.77it/s] 


,Scaffold,Start,End
0,NC_036187.1,11023,11039
1,NC_036187.1,11463,11479
2,NC_036187.1,16433,16449
3,NC_036187.1,17233,17250
4,NC_036187.1,31951,31969
...,...,...,...
86636,NW_019227247.1,576,591
86637,NW_019227261.1,234,247
86638,NW_019227275.1,556,568
86639,NW_019227281.1,479,495


In [26]:
results.to_csv("zdnabert_predictions.bed", index = False, header = False, sep = "\t")

In [1]:
print(1)

1


# ZHUNT

In [2]:
!wget https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/002/706/865/GCF_002706865.2_ASM270686v3/GCF_002706865.2_ASM270686v3_genomic.fna.gz


--2025-06-18 01:49:54--  https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/002/706/865/GCF_002706865.2_ASM270686v3/GCF_002706865.2_ASM270686v3_genomic.fna.gz
Распознаётся ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)… 130.14.250.31, 130.14.250.10, 130.14.250.11, ...
Подключение к ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)|130.14.250.31|:443... соединение установлено.
HTTP-запрос отправлен. Ожидание ответа… 200 OK
Длина: 135791011 (130M) [application/x-gzip]
Сохранение в: «GCF_002706865.2_ASM270686v3_genomic.fna.gz»

GCF_002706865.2_ASM 100%[===================>] 129,50M   193KB/s    за 43m 14s 

2025-06-18 02:33:10 (51,1 KB/s) - «GCF_002706865.2_ASM270686v3_genomic.fna.gz» сохранён [135791011/135791011]



In [3]:
# !cd /kaggle/working
!gunzip GCF_002706865.2_ASM270686v3_genomic.fna.gz
!mv GCF_002706865.2_ASM270686v3_genomic.fna  genomic.fna

In [4]:
!chmod a+x zhunt3-alan.c
!gcc zhunt3-alan.c -lm -o zhunt3 -std=c90 -O3
!./zhunt3 12 8 12 genomic.fna

dinucleotides 12
min/max 8 12
min/max 8 12
operating on genomic.fna
calculating zscore
opening genomic.fna
inputting sequence
opening genomic.fna.Z-SCORE

 run time=18202 sec
use min/max 8 12
analyzing_zscore
opening genomic.fna.Z-SCORE
opening genomic.fna
inputting sequence


In [5]:
!head genomic.fna.Z-SCORE

genomic.fna 429632472 8 12
1 17 16  30.701  27.681 3.501484e-01 nctatatanaacagnt   ASASASASSASASAAS
2 24 22  29.473  23.888 2.112095e+00 ctatatanaacagntgnncaca   SASASAASASASASASASASAS
3 19 16  30.776  20.904 3.398968e-01 tatatanaacagntgn   ASASASSASASAASAS
4 28 24  28.077  29.155 3.558237e+00 atatanaacagntgnncacatcca   SASAASASASASASASASASASAS
5 21 16  31.876  11.465 2.069336e-01 tatanaacagntgnnc   ASASSASASAASASAS
6 28 22  27.775  32.072 4.084399e+00 atanaacagntgnncacatcca   SAASASASASASASASASASAS
7 29 22  31.274  12.028 2.756369e-01 tanaacagntgnncacatccat   ASSASASAASASSASASASASA
8 28 20  27.502  34.450 4.663514e+00 anaacagntgnncacatcca   ASASASASASASASASASAS
9 29 20  30.573  16.961 3.677583e-01 naacagntgnncacatccat   SASASAASASSASASASASA


In [8]:
!pip install biopython
import numpy as np
from Bio import SeqIO
from io import StringIO, BytesIO
from tqdm import tqdm
import scipy
from scipy import ndimage


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [11]:
seg_dict = {}
sum = 1
input_file = "genomic.fna"
fasta_sequence = SeqIO.parse(input_file, "fasta")
for record in SeqIO.parse(input_file, "fasta"):
    name, sequence = record.id, str(record.seq)
    seg_dict[name] = (sum, sum + len(sequence) - 1)
    sum += len(sequence)
seg_dict

{'NC_036187.1': (1, 13549330),
 'NC_036188.1': (13549331, 28980864),
 'NC_036189.1': (28980865, 35445426),
 'NC_036190.1': (35445427, 45767610),
 'NC_036191.1': (45767611, 57665964),
 'NC_036192.1': (57665965, 71128740),
 'NC_036193.1': (71128741, 86614437),
 'NC_036194.1': (86614438, 99036415),
 'NC_036195.1': (99036416, 113973349),
 'NC_036196.1': (113973350, 122536404),
 'NC_036197.1': (122536405, 135837346),
 'NC_036198.1': (135837347, 146314032),
 'NC_036199.1': (146314033, 153666839),
 'NC_036200.1': (153666840, 169307705),
 'NC_036201.1': (169307706, 183233784),
 'NC_036202.1': (183233785, 199505306),
 'NC_036203.1': (199505307, 213097340),
 'NC_036204.1': (213097341, 225289058),
 'NC_036205.1': (225289059, 233482952),
 'NC_036206.1': (233482953, 247111879),
 'NC_036207.1': (247111880, 261320769),
 'NC_036208.1': (261320770, 275592185),
 'NC_036209.1': (275592186, 289472327),
 'NC_036210.1': (289472328, 301674449),
 'NC_036211.1': (301674450, 315357592),
 'NC_036212.1': (3153575

In [12]:
!rm "results_zhunt.bed"

with open("genomic.fna.Z-SCORE", "r") as f:
    ind = -1
    while 1:
        ind += 1
        line = f.readline()
        if not line:
            break
        if ind < 1:
            continue
        start = int(line.split()[0])
        end = int(line.split()[1])
        score = float(line.split()[5])
        name_start = ""
        name_end = ""
        if score <= 400: # фильтрация результатов по z-score
            continue
        for key, (l, r) in seg_dict.items():
            if start >= l and start <= r:
                name_start = key
                start -= l
                if end >= l and end <= r:
                    name_end = key
                    end -= l
                break
        if name_start != name_end:
            continue
        if name_start == "":
            continue
        with open("results_zhunt.bed", "a") as g:
            g.write(f"{name_start}\t{start}\t{end}\n")

!head "results_zhunt.bed"

rm: results_zhunt.bed: No such file or directory


NC_036187.1	11033	11057
NC_036187.1	11035	11059
NC_036187.1	11036	11060
NC_036187.1	11037	11059
NC_036187.1	11038	11060
NC_036187.1	11039	11059
NC_036187.1	11040	11060
NC_036187.1	11041	11059
NC_036187.1	11042	11060
NC_036187.1	11043	11059


# Квадруплексы

In [13]:
import re
with open("g4.bed", "w") as f:
    patterns = [("(G{3,5}[ATGC]{1,7}){3,}G{3,5}", "+"), ("G{3,5}([ATGC]{1,7}G{3,5}){3,}", "-")]
    for part in open("genomic.fna").read().split(">")[1:]:
        name = part.split()[0]
        seq = "".join(part.split('\n')[1:]).upper()
        for pattern, strand in patterns:
            for item in re.finditer(pattern, seq):
                f.write(f"{name}\t{item.start()}\t{item.end()}\n")

In [18]:
!head g4.bed

NC_036187.1	21441	21479
NC_036187.1	75201	75235
NC_036187.1	171729	171756
NC_036187.1	281354	281378
NC_036187.1	295194	295215
NC_036187.1	300136	300164
NC_036187.1	476047	476109
NC_036187.1	558650	558682
NC_036187.1	674684	674726
NC_036187.1	840347	840377


# Строим необходимые таблицы

In [28]:
!wget https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/002/706/865/GCF_002706865.2_ASM270686v3/GCF_002706865.2_ASM270686v3_genomic.gff.gz 
!gunzip GCF_002706865.2_ASM270686v3_genomic.gff.gz 
!mv GCF_002706865.2_ASM270686v3_genomic.gff genomic.gff

--2025-06-18 13:20:20--  https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/002/706/865/GCF_002706865.2_ASM270686v3/GCF_002706865.2_ASM270686v3_genomic.gff.gz
Распознаётся ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)… 130.14.250.31, 130.14.250.7, 130.14.250.12, ...
Подключение к ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)|130.14.250.31|:443... соединение установлено.
HTTP-запрос отправлен. Ожидание ответа… 200 OK
Длина: 6129584 (5,8M) [application/x-gzip]
Сохранение в: «GCF_002706865.2_ASM270686v3_genomic.gff.gz»

GCF_002706865.2_ASM 100%[===================>]   5,84M  75,0KB/s    за 53s     

2025-06-18 13:21:13 (113 KB/s) - «GCF_002706865.2_ASM270686v3_genomic.gff.gz» сохранён [6129584/6129584]



In [29]:
feature_types = set()
for line in open("genomic.gff"):
    if line[0] == "#":
        continue
    feature_types.add(line.split()[2])

In [30]:
feature_types

{'CDS',
 'cDNA_match',
 'exon',
 'gene',
 'lnc_RNA',
 'mRNA',
 'pseudogene',
 'rRNA',
 'region',
 'sequence_feature',
 'tRNA',
 'transcript'}

In [31]:
part_begin_pos = [0]
part_name = []
for part in open("genomic.fna").read().split(">")[1:]:
    name = part.split()[0]
    seq = "".join(part.split("\n")[1:]).upper()
    part_name.append(name)
    part_begin_pos.append(part_begin_pos[-1] + len(seq))

In [32]:
with open("genes.bed", "w") as f:
    for line in open("genomic.gff"):
        if line[0] == "#":
            continue
        if line.split()[2] == "gene":
            start, end = map(int, line.split()[3:5])
            name = line.split()[0]
            f.write(f"{name}\t{start}\t{end}\n")
with open("exons.bed", "w") as f:
    for line in open("genomic.gff"):
        if line[0] == "#":
            continue
        if line.split()[2] == "exon":
            start, end = map(int, line.split()[3:5])
            name = line.split()[0]
            f.write(f"{name}\t{start}\t{end}\n")
with open("lengths", "w") as f:
    for i in range(len(part_name)):
        print(part_name[i], part_begin_pos[i + 1] - part_begin_pos[i], file = f, sep = "\t")

In [35]:
!brew install bedtools

==> Auto-updating Homebrew...
Adjust how often this is run with HOMEBREW_AUTO_UPDATE_SECS or disable with
HOMEBREW_NO_AUTO_UPDATE. Hide these hints with HOMEBREW_NO_ENV_HINTS (see `man brew`).
==> Downloading https://ghcr.io/v2/homebrew/portable-ruby/portable-ruby/blobs/sha256:fd162df7a06190ee800a9e6afd28f4466d33548821a480ba043cd927b44d60f7
######################################################################### 100.0%                                        17.0%                      20.5%                                                        21.8%                                                    22.4%                                           22.6%                                             23.4%                                                23.5%                                               24.8%                       27.3%                             27.6%                            28.0%                                        28.2%                                            

In [36]:
!bedtools subtract -a genes.bed -b exons.bed > introns.bed

In [38]:
!bedtools flank -l 1000 -r 0 -i genes.bed -g lengths > upstream.bed
!bedtools flank -r 200 -l 0 -i genes.bed -g lengths > downstream.bed
!bedtools complement -i genes.bed -g lengths > intergenic.bed


In [40]:
types = ["g4", "zhunt", "zdnabert"]
locs = ["genes", "introns", "upstream", "downstream", "intergenic"]
df = pd.DataFrame(
    columns=list(map(lambda s: s + " count", types)) + \
    list(map(lambda s: s + " %", types)),
    index=locs
)
for loc in tqdm(locs):
    for ty in tqdm(types, leave=False):
        filename = "zdnabert_predictions.bed"
        if ty == "g4":
            filename = "g4.bed"
        if ty == "zhunt":
            filename = "results_zhunt.bed"
        all_count = len(open(filename).readlines())
        !bedtools intersect -wa -b {loc + ".bed"} -a {filename} | uniq > tmp
        in_count = len(open("tmp").readlines())
        df.loc[loc, ty + " count"] = in_count
        df.loc[loc, ty + " %"] = in_count / all_count

100%|██████████| 5/5 [00:07<00:00,  1.41s/it]


In [41]:
df

,g4 count,zhunt count,zdnabert count,g4 %,zhunt %,zdnabert %
genes,12122,994127,56985,0.563709,0.637476,0.657714
introns,10999,823574,35848,0.511486,0.52811,0.413753
upstream,632,53356,3341,0.02939,0.034214,0.038561
downstream,97,15718,1065,0.004511,0.010079,0.012292
intergenic,9248,567292,29920,0.43006,0.363772,0.345333


In [42]:
df = pd.DataFrame(
    columns=list(map(lambda s: s + " region count", types)) + \
    list(map(lambda s: s + " region %", types)),
    index=locs
)
for loc in tqdm(locs):
    for ty in tqdm(types, leave=False):
        filename = "zdnabert_predictions.bed"
        if ty == "g4":
            filename = "g4.bed"
        if ty == "zhunt":
            filename = "results_zhunt.bed"
        all_count = len(open(loc + ".bed").readlines())
        !bedtools intersect -wa -a {loc + ".bed"} -b {filename} | uniq > tmp
        in_count = len(open("tmp").readlines())
        df.loc[loc, ty + " region count"] = in_count
        df.loc[loc, ty + " region %"] = in_count / all_count

100%|██████████| 5/5 [00:05<00:00,  1.18s/it]


In [43]:
df

,g4 region count,zhunt region count,zdnabert region count,g4 region %,zhunt region %,zdnabert region %
genes,3046,10416,8279,0.193274,0.660914,0.525317
introns,4224,28049,13942,0.041587,0.276157,0.137266
upstream,322,4028,2392,0.020431,0.255584,0.151777
downstream,54,1228,839,0.003427,0.077934,0.053246
intergenic,2424,8429,6335,0.154948,0.538801,0.404948
